# KENV

<a href=mailto:fuodorov1998@gmail.com>V. Fedorov</a>, <a href=mailto:nikdanila@bk.ru>D. Nikiforov</a>, <a href=http://www.inp.nsk.su/~petrenko/>A. Petrenko</a>, (Novosibirsk, 2019)

## Manual

In [1]:
import kenv as kv

Сan call for help:

In [2]:
help(kv)

Help on package kenv:

NAME
    kenv

PACKAGE CONTENTS
    accelerator
    beam
    solver

CLASSES
    builtins.object
        kenv.accelerator.Accelerator
        kenv.accelerator.Element
        kenv.beam.Beam
        kenv.beam.Particle
        kenv.solver.Simulation
    
    class Accelerator(builtins.object)
     |  Accelerator(start: float, stop: float, step: float)
     |  
     |  Create an accelerator.
     |  
     |  Create an accelerator with parameters:
     |  start [m] --- the beginning of the accelerator,
     |  stop [m] --- end of the accelerator,
     |  step [m] --- step method along the accelerator.
     |  
     |  Can add a solenoids, quadrupoles and accelerating modules.
     |  
     |  Methods defined here:
     |  
     |  __init__(self, start: float, stop: float, step: float)
     |      Initialize self.  See help(type(self)) for accurate signature.
     |  
     |  __str__(self)
     |      Return str(self).
     |  
     |  add_acc = add_accel(self, name: 

## Simulation

To obtain an envelope, you need to create a beam and an accelerator. Then load the simulation of this beam in this accelerator and apply tracking.

In [3]:
#help(kv.Beam)

In [4]:
beam = kv.Beam(energy=2,
               current=2e3,
               radius=50e-3,
               angular=.0e-3,
               normalized_emittans=1000e-6)

In [5]:
beam.add_particle('Center',50e-3,.0e-3, 0.0e-3,0)

Can print:

In [6]:
print(beam)

Beam parameters:
	Current	2000.0 A
	Energy	2.0 MeV
	Radius x	50.0 mm
	Radius y	50.0 mm
	Angular x	0.0 mrad
	Angular y	0.0 mrad
	Normalized emittans x	1000.0 mm*mrad
	Normalized emittans y	1000.0 mm*mrad
	Particles:
	[ 50.0 mm, 0.0 mm, 0.0 mrad, 0.0 mrad, 0.0 rad, 'Center'],



In [7]:
#help(kv.Accelerator)

In [8]:
accelerator = kv.Accelerator(0, 5, 0.001)

In [9]:
accelerator.add_new_solenoid('Sol. 1', 1, 0.05, 'notebooks/input/B_z(z).dat')
#accelerator.add_sol('Sol. 1', 1, 0.05, 'B_z(z).dat')
accelerator.add_new_accel('Acc. 1', 2.5, -1, 'notebooks/input/E_z(z).dat')
#accelerator.add_acc('Sol. 1', 1, 0.05, 'B_z(z).dat')

In [10]:
print(accelerator)

Accelerator structure.
	Solenoids:
	[ 1.00000 m, 0.05000 T, 'notebooks/input/B_z(z).dat', 'Sol. 1'],
	Accelerating modules:
	[ 2.50000 m, -1.00000 Mv/m, 'notebooks/input/E_z(z).dat', 'Acc. 1'],
	Quadrupoles:



In [11]:
Solenoids = [ 
    [ 2.0000, 0.06, 'notebooks/input/B_z(z).dat', 'Sol. 2'],
    [ 3.0000, 0.07, 'notebooks/input/B_z(z).dat', 'Sol. 3'],
    [ 4.0000, 0.08, 'notebooks/input/B_z(z).dat', 'Sol. 4'],
]

In [12]:
for   z0, B0, filename, name in Solenoids:
    accelerator.Bz_beamline[name] = kv.Element(z0, B0, filename, name)

In [13]:
print(accelerator)

Accelerator structure.
	Solenoids:
	[ 1.00000 m, 0.05000 T, 'notebooks/input/B_z(z).dat', 'Sol. 1'],
	[ 2.00000 m, 0.06000 T, 'notebooks/input/B_z(z).dat', 'Sol. 2'],
	[ 3.00000 m, 0.07000 T, 'notebooks/input/B_z(z).dat', 'Sol. 3'],
	[ 4.00000 m, 0.08000 T, 'notebooks/input/B_z(z).dat', 'Sol. 4'],
	Accelerating modules:
	[ 2.50000 m, -1.00000 Mv/m, 'notebooks/input/E_z(z).dat', 'Acc. 1'],
	Quadrupoles:



In [ ]:
accelerator.compile()

In [ ]:
#help(kv.Simulation)

In [ ]:
simulation = kv.Simulation(beam, accelerator)

In [ ]:
simulation.track(solver='odeint', rtol=1e-5)

In [ ]:
#simulation.envelope_x
#simulation.particle_phase['Center']

## Graphic with holoviews

### matplotlib

In [ ]:
import holoviews as hv
hv.extension('matplotlib')

%opts Layout [tight=True]
%output size=150 backend='matplotlib' fig='png'

%opts Area Curve [aspect=3 show_grid=True]
%opts Area  (alpha=0.25)
%opts Curve (alpha=0.5)
%opts Area.Beam [aspect=3 show_grid=True] (color='red' alpha=0.3)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
dim_z  = hv.Dimension('z',  unit='m', range=(accelerator.start, accelerator.stop))
dim_Bz = hv.Dimension('Bz', unit='T', label='Bz')
dim_Ez = hv.Dimension('Ez', unit='MV/m', label='Ez')

dim_r = hv.Dimension('r', label="Beam r", unit='mm')
dim_phase = hv.Dimension('$\phi$', label="Larmor phase", unit='rad')


In [ ]:
z_Ez = hv.Area((accelerator.parameter,accelerator.Ez(accelerator.parameter)), kdims=dim_z, vdims=dim_Ez)
z_Bz = hv.Area((accelerator.parameter,accelerator.Bz(accelerator.parameter)), kdims=dim_z, vdims=dim_Bz)
z_r = hv.Area(((accelerator.parameter,simulation.envelope_x*1e3)), kdims=[dim_z], vdims=[dim_r], group='Beam')
z_x_particle = hv.Curve(((accelerator.parameter,simulation.particle_x['Center']*1e3)), kdims=[dim_z], vdims=[dim_r], group='Beam')
#z_y_particle = hv.Curve(((accelerator.parameter,simulation.particle_y['Center']*1e3)), kdims=[dim_z], vdims=[dim_r], group='Beam')
z_phase = hv.Curve(((accelerator.parameter,simulation.particle_phase['Center'])), kdims=[dim_z], vdims=[dim_phase], group='Beam')

(z_r*z_x_particle+z_phase+z_Bz+z_Ez).cols(1)


In [ ]:
dim_dEzdz = hv.Dimension('dEzdz', unit='$\mathrm{MV/m^2}$', label='$dE_z/dz$', range=(-.50,+.50))
dim_z  = hv.Dimension('z',  unit='m', range=(2, 3))
k=1
dEzdz = (accelerator.Ez(accelerator.parameter+k*accelerator.step) - accelerator.Ez(accelerator.parameter-k*accelerator.step))/accelerator.step/k
hv.Area((accelerator.parameter,dEzdz), kdims=dim_z, vdims=dim_dEzdz)